In [ ]:
#Alexa Kozachok

In [ ]:
# Task 5

In [1]:
import random

class Alexa_Hamming:
    #using standart Python integer type

    @staticmethod
    def get_random_vector_of_minusOneAndOnes(xSize = 16):
        return [random.randrange(-1, 2, 2) for xIndex in range(xSize)]
    
    @staticmethod
    def set_of_types(*args):
        xSet_of_types = set([])
        for arg in args:
            xSet_of_types.add(type(arg))
        return xSet_of_types
    
    @staticmethod
    def is_iterable_object(arg):
        try:
            iterator = iter(arg)
            #print(arg, 'is iterable')
            return True
        except TypeError as iterator_error:
            #print(arg, 'is not iterable')
            return False
        
    @staticmethod
    def is_iterable(*args):
        for arg in args:
            if Alexa_Hamming.is_iterable_object(arg):
                pass
            else:
                return False
        return True
        
    @staticmethod
    def is_size_the_same(*args):
        xSet_of_sizes = set([])
        for arg in args:
            xSet_of_sizes.add(len(arg))
        if 1 == len(xSet_of_sizes):
            return True
        else:
            return False

    @staticmethod
    def inner_product(xVec1, xVec2):
        xSigned_inner_product = 0
        for i in range(len(xVec1)):
            xSigned_inner_product += (xVec1[i]*xVec2[i])
        return xSigned_inner_product
    
    @staticmethod
    def different_bits_via_inner_product(xVec1, xVec2):
        return ((len(xVec1) - Alexa_Hamming.inner_product(xVec1, xVec2)) // 2)
    
    @staticmethod
    def hamming_distance(xVec1, xVec2, xType = list):
        assert Alexa_Hamming.is_iterable(xVec1, xVec2), \
        "at least one of arg is not iterable object (could not be a vector)"
        assert set([xType]) == Alexa_Hamming.set_of_types(xVec1, xVec2), \
        "objects are of different types or function needs other types of vectors"
        assert Alexa_Hamming.is_size_the_same(xVec1, xVec2), \
        "the size (length) of args (vectors) are not the same"
        xSize = len(xVec1)
        return (Alexa_Hamming.different_bits_via_inner_product(xVec1, xVec2)/xSize)

    @staticmethod
    def hamming_distance_typeList_fast(xVec1, xVec2, xSize = 16):
        return ((xSize - sum([xVec1[i]*xVec2[i] for i in range(xSize)])) / (2*xSize))
    
    @staticmethod
    def hamming_distance_typeList_length16_fast(xVec1, xVec2):
        return ((16 - sum([xVec1[i]*xVec2[i] for i in range(16)])) / 32)

In [2]:
import sys
sys.path.append("c:\knu\piton4")
from bn256 import *

In [3]:
# Algorithm from TASK 4 (TASK 3)

import collections
mskT = collections.namedtuple('mskT',     ['h_g1', 's_vector', 't_vector', 'h_g2', 'u_vector', 'v_vector'])
#==============================================
class Alexa_IPE_Setup:
    
    def __init__(self, n, g1, g2, h1, h2, inv_g1 = None, inv_g2 = None):
        self.n  = n
        self.G1 = g1
        self.G2 = g2
        self.H1 = h1
        self.H2 = h2
        self.invG1 = inv_g1
        self.invG2 = inv_g2
        self.msk = Alexa_IPE_Setup.get_msk(self.n, self.G1, self.G2, self.H1, self.H2)
    
    @staticmethod
    def get_msk(n, g1, g2, h1, h2):
        s_vector = [ rand_elem() for i in range(n) ]
        t_vector = [ rand_elem() for i in range(n) ]
        u_vector = [ rand_elem() for i in range((n+2)) ]
        v_vector = [ rand_elem() for i in range((n+2)) ]
        gen1_h = [ g1_add(G1.scalar_mul(s_vector[i]), h1.scalar_mul(t_vector[i])) for i in range(n) ]
        gen2_h = [ g2_add(G2.scalar_mul(u_vector[i]), h2.scalar_mul(v_vector[i])) for i in range((n+2)) ]
        return mskT._make((gen1_h, s_vector, t_vector, gen2_h, u_vector, v_vector))
    
    @staticmethod
    def oneToGminusOneToInvG(xOneminusOne, G, invG):
        assert xOneminusOne in [-1,1], " def oneToG1minusOneToInvG1(xOneminusOne, G, invG): argument type|range ERROR "
        if -1 == xOneminusOne:
            return invG
        else:
            return G
        
    @staticmethod
    def vectorOfNumberToVectorOfPoints(xVector, G, invG):
        assert set(xVector).issubset(set([-1,1])), " ERROR "
        return [ Alexa_IPE_Setup.oneToGminusOneToInvG(xElement, G, invG) for xElement in xVector ]
    
#==============================================

class Alexa_IPE_Registration:
    
    def __init__(self, msk, c_vector, n):
        self.msk = msk
        self.c = c_vector
        self.n = n
        self.r0 = rand_elem()
        self.reg_template = []
        self.reg_template.append(G2.scalar_mul(self.r0))
        self.reg_template.append(H2.scalar_mul(self.r0))
        self.h2 = self.msk.h_g2
        self.reg_template += [xElement.scalar_mul(self.r0) for xElement in self.h2 ]
        #
        self.powers = self.msk.s_vector
        self.ProductTemp = \
        Alexa_IPE_Registration.akaProduct_g2(self.c, self.powers, 0, n, 0, 0)
        self.reg_template[2] = g2_add(self.reg_template[2], self.ProductTemp.scalar_mul(order-1))
        #
        self.powers = self.msk.t_vector
        self.ProductTemp = \
        Alexa_IPE_Registration.akaProduct_g2(self.c, self.powers, 0, n, 0, 0)
        self.reg_template[3] = g2_add(self.reg_template[3], self.ProductTemp.scalar_mul(order-1))
        #
        for i in range(0, self.n, 1):
            self.reg_template[i+4] = g2_add(self.reg_template[i+4], self.c[i])
           
    @staticmethod            
    def akaProduct_g2(c_vector, power_vector, j, n, shift_c_index = 0, shift_power_index = 0):
        xAkaProduct = c_vector[j + shift_c_index].scalar_mul(power_vector[j + shift_power_index])
        for i in range((1+j), n, 1):
            temp = c_vector[i + shift_c_index].scalar_mul(power_vector[i + shift_power_index])
            xAkaProduct = g2_add(xAkaProduct, temp)
            del temp
        return xAkaProduct
    
#==============================================
class Alexa_IPE_Authentication:
    
    def __init__(self, msk, c_vector, n):
        self.msk = msk
        self.c = c_vector
        self.n = n
        self.r0 = rand_elem()
        self.auth_template = []
        self.G1zero = G1.scalar_mul(0)
        self.auth_template.append(self.G1zero)
        self.auth_template.append(self.G1zero)
        self.auth_template.append(G1.scalar_mul(self.r0))
        self.auth_template.append(H1.scalar_mul(self.r0))
        #
        self.h1 = self.msk.h_g1
        self.auth_template += [g1_add(self.h1[i].scalar_mul(self.r0), self.c[i]) for i in range(self.n) ]
        #
        self.powers = self.msk.u_vector
        self.ProductTemp = \
        Alexa_IPE_Authentication.akaProduct_g1(self.auth_template, self.powers, 0, (self.n + 2), 2, 0)
        self.auth_template[0] = self.ProductTemp.scalar_mul(order-1)
        #
        self.powers = self.msk.v_vector
        self.ProductTemp = \
        Alexa_IPE_Authentication.akaProduct_g1(self.auth_template, self.powers, 0, (self.n + 2), 2, 0)
        self.auth_template[1] = self.ProductTemp.scalar_mul(order-1)

    @staticmethod            
    def akaProduct_g1(c_vector, power_vector, j, n, shift_c_index = 0, shift_power_index = 0):
        xAkaProduct = c_vector[j + shift_c_index].scalar_mul(power_vector[j + shift_power_index])
        for i in range((1+j),n, 1):
            temp = c_vector[i + shift_c_index].scalar_mul(power_vector[i + shift_power_index])
            xAkaProduct = g1_add(xAkaProduct, temp)
            del temp
        return xAkaProduct
#==============================================
class Alexa_IPE_Decryption:
    
    def __init__(self, pairing, g1, g2, reg_template, auth_template):
        assert len(auth_template) == len(reg_template) , " ERORR : templates of different length "
        self.n  = len(auth_template)
        self.k = (self.n - 4)
        self.G1 = g1
        self.G2 = g2
        self.pairing = pairing
        self.reg_template  = reg_template
        self.auth_template = auth_template
        self.pairingG2G1 = self.pairing(self.G2, self.G1)
        self.pairingOfTemplates = None
        self.inner_product = None
        self.hamming_dictance = None

    def get_inner_product(self):
        assert self.pairingOfTemplates is not None
        flag = True
        xPower = 0
        gt_temp     = gt_scalar_mult(self.pairingG2G1, xPower)
        if  gt_temp == self.pairingOfTemplates:
            self.inner_product =  xPower
        else:
            xPower += 2
            gt_temp2    = gt_scalar_mult(self.pairingG2G1, 2)
            gt_temp     = gt_temp2
            gt_temp2Inv = gt_temp2.inverse()
            gt_tempInv  = gt_temp2Inv
            while flag:
                if self.k < xPower:
                    flag = False
                else:
                    if  gt_tempInv == self.pairingOfTemplates:
                        self.inner_product = -xPower
                        flag = False
                    elif gt_temp == self.pairingOfTemplates:
                        self.inner_product =  xPower
                        flag = False
                    else:
                        gt_temp    = gt_add(gt_temp, gt_temp2)
                        gt_tempInv = gt_add(gt_tempInv, gt_temp2Inv)
                        xPower += 2
                        pass

    def get_hamming_dictance(self):
        assert self.inner_product is not None
        self.hamming_dictance = ((self.k - self.inner_product) / (2*self.k))
                    
    def get_pairing_of_templates(self):
        self.pairingOfTemplates = Alexa_IPE_Decryption.get_pairing_of_templates_staticmethod(self.pairing, self.reg_template, self.auth_template)
                
    @staticmethod
    def get_pairing_of_templates_staticmethod(pairing, dk_vector, ct_vector):
        assert len(dk_vector) == len(ct_vector) , " ERORR : templates of different length "
        n = len(dk_vector)
        pairing_list = [pairing(dk_vector[i], ct_vector[i]) \
                  for i in range(n)]
        product = pairing_list[0]
        for i in range(1,n, 1):
            product = gt_add(product, pairing_list[i])
        del pairing_list
        return product
#==============================================

In [17]:
# Algorithm for TASK 5
# has some speed up
#==============================================
class Goder:
    
    def __init__(self, x, p = order):
        self.p = p
        self.x = x
        self.xmod()

    def __repr__(self):
        return "{}".format(self.x)
    
    def value(self):
        self.xmod()
        return self.x

    def xmod(self):
        self.x = (self.x%self.p)
        if 0 > self.x:
            self.x = (self.x + self.p)
            self.x = (self.x%self.p)
            
    def __add__(self, other):
        res = (self.x + other.x)
        return Goder(res)

    def __sub__(self, other):
        res = (self.x - other.x)
        return Goder(res)

    def __mul__(self, other):
        res = (self.x*other.x)
        return Goder(res)
#==============================================
#==============================================
import collections
mskT2 = collections.namedtuple('mskT2',  ['g1', 'inv_g1', 's_vector', 't_vector', 'g2', 'inv_g2', 'u_vector', 'v_vector', 'alpha', 'beta'])
#==============================================
import copy

class Alexa2_IPE_Setup:
    
    def __init__(self, n, g1, g2): #(curve_G, twist_G)
        self.n  = n
        self.G1 = g1
        self.G2 = g2
        self.invG1 = self.G1.scalar_mul((order-1))
        self.invG2 = copy.deepcopy(self.G2)
        self.invG2.negate()
        self.alpha = Goder(rand_elem())
        self.beta  = Goder(rand_elem())
        self.msk = Alexa2_IPE_Setup.get_msk(self.n, self.G1, self.invG1, self.G2, self.invG2, \
                                            self.alpha, self.beta)
    
    @staticmethod
    def get_msk(n, g1, inv_g1, g2, inv_g2, alpha, beta):
        s_vector = [ Goder(rand_elem()) for i in range(n) ]
        t_vector = [ Goder(rand_elem()) for i in range(n) ]
        u_vector = [ Goder(rand_elem()) for i in range((n+2)) ]
        v_vector = [ Goder(rand_elem()) for i in range((n+2)) ]
        return mskT2._make((g1, inv_g1, s_vector, t_vector, g2, inv_g2, u_vector, v_vector, alpha, beta))
        
    @staticmethod
    def oneToGminusOneToInvG(xOneminusOne):
        assert xOneminusOne in [-1,1], " def oneToG1minusOneToInvG1(xOneminusOne, G, invG): argument type|range ERROR "
        if -1 == xOneminusOne:
            return Goder(xOneminusOne)
        else:
            return Goder(xOneminusOne)
        
    @staticmethod
    def vectorOfNumberToVectorOfPoints(xVector):
        return [ Alexa2_IPE_Setup.oneToGminusOneToInvG(xElement) for xElement in xVector ] 
    
    @staticmethod            
    def akaProduct(c_vector, power_vector, n):
        xAkaProduct = (c_vector[0]*power_vector[0])
        for i in range(1, n, 1):
            temp = (c_vector[i]*power_vector[i])
            xAkaProduct = (xAkaProduct + temp)
        return xAkaProduct

#==============================================

class Alexa2_IPE_Registration:
    
    def __init__(self, msk, c_vector, n):
        r0 = Goder(rand_elem())
        reg_template_tmp = ([r0, r0*msk.beta, \
                              (r0*(msk.u_vector[0] + msk.beta*msk.v_vector[0])), (r0*(msk.u_vector[1] + msk.beta*msk.v_vector[1]))] + \
                             [ (r0*(msk.u_vector[(i+2)] + msk.beta*msk.v_vector[(i+2)]) + c_vector[i]) for i in range(n) ])
        #
        self.ProductTemp = \
        Alexa2_IPE_Setup.akaProduct(c_vector, msk.s_vector, n)
        reg_template_tmp[2] = (reg_template_tmp[2] - self.ProductTemp)
        #
        self.ProductTemp = \
        Alexa2_IPE_Setup.akaProduct(c_vector, msk.t_vector, n)
        reg_template_tmp[3] = (reg_template_tmp[3] - self.ProductTemp)
        #
        self.reg_template = [ msk.g2.scalar_mul(xEl.value()) for xEl in reg_template_tmp ]
        del reg_template_tmp
    
#==============================================
    
class Alexa2_IPE_Authentication:
    
    def __init__(self, msk, c_vector, n):
        r1 = Goder(rand_elem())
        auth_template_tmp = ([Goder(0), Goder(0), r1, r1*msk.alpha] + \
                             [ (r1*(msk.s_vector[i] + msk.alpha*msk.t_vector[i]) + c_vector[i]) for i in range(n) ])
        #
        auth_template_tmp2 = auth_template_tmp[2:]
        ProductTemp = \
        Alexa2_IPE_Setup.akaProduct(auth_template_tmp2, msk.u_vector, (n + 2))
        auth_template_tmp[0] -= ProductTemp
        ProductTemp = \
        Alexa2_IPE_Setup.akaProduct(auth_template_tmp2, msk.v_vector, (n + 2))
        auth_template_tmp[1] -= ProductTemp
        del auth_template_tmp2
        self.auth_template = [ msk.g1.scalar_mul(xEl.value()) for xEl in auth_template_tmp ]
        del auth_template_tmp
        
#==============================================
class Alexa2_IPE_Decryption:
    
    def __init__(self, pairing, g1, g2, reg_template, auth_template):
        assert len(auth_template) == len(reg_template) , " ERORR : templates of different length "
        self.n  = len(auth_template)
        self.k = (self.n - 4)
        self.G1 = g1
        self.G2 = g2
        self.pairing = pairing
        self.reg_template  = reg_template
        self.auth_template = auth_template
        self.pairingG2G1 = self.pairing(self.G2, self.G1)
        self.pairingOfTemplates = None
        self.inner_product = None
        self.hamming_dictance = None

    def get_inner_product(self):
        assert self.pairingOfTemplates is not None
        flag = True
        xPower = 0
        gt_temp     = gt_scalar_mult(self.pairingG2G1, xPower)
        if  gt_temp == self.pairingOfTemplates:
            self.inner_product =  xPower
        else:
            xPower += 2
            gt_temp2    = gt_scalar_mult(self.pairingG2G1, 2)
            gt_temp     = gt_temp2
            gt_temp2Inv = gt_temp2.inverse()
            gt_tempInv  = gt_temp2Inv
            while flag:
                if self.k < xPower:
                    flag = False
                else:
                    if  gt_tempInv == self.pairingOfTemplates:
                        self.inner_product = -xPower
                        flag = False
                    elif gt_temp == self.pairingOfTemplates:
                        self.inner_product =  xPower
                        flag = False
                    else:
                        gt_temp    = gt_add(gt_temp, gt_temp2)
                        gt_tempInv = gt_add(gt_tempInv, gt_temp2Inv)
                        xPower += 2
                        pass

    def get_hamming_dictance(self):
        assert self.inner_product is not None
        self.hamming_dictance = ((self.k - self.inner_product) / (2*self.k))
                    
    def get_pairing_of_templates(self):
        self.pairingOfTemplates = Alexa2_IPE_Decryption.get_pairing_of_templates_staticmethod(self.pairing, self.reg_template, self.auth_template)
                
    @staticmethod
    def get_pairing_of_templates_staticmethod(pairing, dk_vector, ct_vector):
        assert len(dk_vector) == len(ct_vector) , " ERORR : templates of different length "
        n = len(dk_vector)
        pairing_list = [pairing(dk_vector[i], ct_vector[i]) \
                  for i in range(n)]
        product = pairing_list[0]
        for i in range(1,n, 1):
            product = gt_add(product, pairing_list[i])
        del pairing_list
        return product
#==============================================

In [5]:
CONST_LENGTH_N = 16
#==============================================
v1 = Alexa_Hamming.get_random_vector_of_minusOneAndOnes(CONST_LENGTH_N)
v2 = Alexa_Hamming.get_random_vector_of_minusOneAndOnes(CONST_LENGTH_N)
print(v1)
print(v2)
print(" inner product via initial vectors : ")
print(Alexa_Hamming.inner_product(v1, v2))
print(" inner product via hamming distance : ")
print(Alexa_Hamming.hamming_distance(v1, v2))

[-1, -1, 1, -1, 1, -1, -1, -1, -1, -1, 1, -1, 1, 1, -1, -1]
[1, 1, -1, -1, -1, 1, -1, 1, 1, 1, 1, -1, 1, -1, -1, 1]
 inner product via initial vectors : 
-4
 inner product via hamming distance : 
0.625


In [10]:
G1 = copy.deepcopy(g1_scalar_base_mult(1))
G2 = copy.deepcopy(g2_scalar_base_mult(1))
invG1  = G1.scalar_mul(gfp_1(order-1).value())
invG2 = copy.deepcopy(G2)
invG2.negate()
alpha = gfp_1(rand_elem())
beta  = gfp_1(rand_elem())
H1 = copy.deepcopy(g1_scalar_base_mult(alpha.value()))
H2 = copy.deepcopy(g2_scalar_base_mult(beta.value()))
#========================================
#========================================
import datetime

time_start = datetime.datetime.now()
#==============================================
time_start_Alexa_IPE_Setup = datetime.datetime.now()
xAlexa_IPE_Setup = Alexa_IPE_Setup(len(v1), G1, G2, H1, H2, invG1, invG2)
time_end_Alexa_IPE_Setup = datetime.datetime.now()
timedelta_Alexa_IPE_Setup = (time_end_Alexa_IPE_Setup - time_start_Alexa_IPE_Setup)
#==============================================
v1_as_points = Alexa_IPE_Setup.vectorOfNumberToVectorOfPoints(v1, xAlexa_IPE_Setup.G2, xAlexa_IPE_Setup.invG2)
v2_as_points = Alexa_IPE_Setup.vectorOfNumberToVectorOfPoints(v2, xAlexa_IPE_Setup.G1, xAlexa_IPE_Setup.invG1)
#==============================================
time_start_xAlexa_IPE_Registration = datetime.datetime.now()
xAlexa_IPE_Registration = \
Alexa_IPE_Registration(xAlexa_IPE_Setup.msk, Alexa_IPE_Setup.vectorOfNumberToVectorOfPoints(v1, G2, invG2), len(v1))
assert len(xAlexa_IPE_Registration.reg_template) == (4 + len(v1)), " something wrong "
time_end_xAlexa_IPE_Registration = datetime.datetime.now()
timedelta_xAlexa_IPE_Registration = \
(time_end_xAlexa_IPE_Registration - time_start_xAlexa_IPE_Registration)
#==============================================
time_start_xAlexa_IPE_Authentication = datetime.datetime.now()
xAlexa_IPE_Authentication = \
Alexa_IPE_Authentication(xAlexa_IPE_Setup.msk, Alexa_IPE_Setup.vectorOfNumberToVectorOfPoints(v2, G1, invG1), len(v2))
assert len(xAlexa_IPE_Authentication.auth_template) == (4 + len(v2)), " something wrong "
assert len(xAlexa_IPE_Authentication.auth_template) == len(xAlexa_IPE_Registration.reg_template), " something wrong "
time_end_xAlexa_IPE_Authentication = datetime.datetime.now()
timedelta_xAlexa_IPE_Authentication = \
(time_end_xAlexa_IPE_Authentication - time_start_xAlexa_IPE_Authentication)
#==============================================
time_start_xAlexa_IPE_Decryption_init = datetime.datetime.now()
xAlexa_IPE_Decryption = \
Alexa_IPE_Decryption(optimal_ate, G1, G2, xAlexa_IPE_Registration.reg_template, xAlexa_IPE_Authentication.auth_template)
time_end_xAlexa_IPE_Decryption_init = datetime.datetime.now()
timedelta_xAlexa_IPE_Decryption_init = \
(time_end_xAlexa_IPE_Decryption_init - time_start_xAlexa_IPE_Decryption_init)

time_start_Alexa_IPE_Decryption_get_pairing_of_templates = datetime.datetime.now()
xAlexa_IPE_Decryption.get_pairing_of_templates()
time_end_Alexa_IPE_Decryption_get_pairing_of_templates = datetime.datetime.now()
timedelta_Alexa_IPE_Decryption_get_pairing_of_templates = \
(time_end_Alexa_IPE_Decryption_get_pairing_of_templates - time_start_Alexa_IPE_Decryption_get_pairing_of_templates)

time_start_Alexa_IPE_Decryption_get_inner_product = datetime.datetime.now()
xAlexa_IPE_Decryption.get_inner_product()
time_end_Alexa_IPE_Decryption_get_inner_product = datetime.datetime.now()
timedelta_Alexa_IPE_Decryption_get_inner_product = \
(time_end_Alexa_IPE_Decryption_get_inner_product - time_start_Alexa_IPE_Decryption_get_inner_product)

time_start_Alexa_IPE_Decryption_get_hamming_dictance = datetime.datetime.now()
xAlexa_IPE_Decryption.get_hamming_dictance()
time_end_Alexa_IPE_Decryption_get_hamming_dictance = datetime.datetime.now()
timedelta_Alexa_IPE_Decryption_get_hamming_dictance = \
(time_end_Alexa_IPE_Decryption_get_hamming_dictance - time_start_Alexa_IPE_Decryption_get_hamming_dictance)
timedelta_xAlexa_IPE_Decryption = \
(timedelta_xAlexa_IPE_Decryption_init + timedelta_Alexa_IPE_Decryption_get_pairing_of_templates + \
timedelta_Alexa_IPE_Decryption_get_inner_product + timedelta_Alexa_IPE_Decryption_get_hamming_dictance)
#==============================================
time_end = datetime.datetime.now()
timedelta = (time_end - time_start)

In [12]:
print(" inner product via IPE : ")
print(xAlexa_IPE_Decryption.inner_product)
print(" hamming dictance via IPE : ")
print(xAlexa_IPE_Decryption.hamming_dictance)
print(" inner product via initial vectors : ")
print(Alexa_Hamming.inner_product(v1, v2))
print(" inner product via hamming distance : ")
print(Alexa_Hamming.hamming_distance(v1, v2))

 inner product via IPE : 
-4
 hamming dictance via IPE : 
0.625
 inner product via initial vectors : 
-4
 inner product via hamming distance : 
0.625


In [13]:
print(timedelta.total_seconds()/60, " (minutes)")
print(timedelta.total_seconds(), " (seconds)")
print((10**6)*timedelta.total_seconds(), " (microseconds)")

0.48482773333333334  (minutes)
29.089664  (seconds)
29089664.0  (microseconds)


In [19]:
#import datetime
#==============================================
time_start2 = datetime.datetime.now()
#==============================================
time_start_Alexa2_IPE_Setup = datetime.datetime.now()
xAlexa2_IPE_Setup = Alexa2_IPE_Setup(len(v1), curve_G, twist_G)
time_end_Alexa2_IPE_Setup = datetime.datetime.now()
timedelta_Alexa2_IPE_Setup = (time_end_Alexa2_IPE_Setup - time_start_Alexa2_IPE_Setup)
#print(timedelta_Alexa2_IPE_Setup.total_seconds())
#==============================================
#v1_as_points2 = Alexa2_IPE_Setup.vectorOfNumberToVectorOfPoints(v1)
#==============================================
time_start_xAlexa2_IPE_Registration = datetime.datetime.now()
xAlexa2_IPE_Registration = \
Alexa2_IPE_Registration(xAlexa2_IPE_Setup.msk, Alexa2_IPE_Setup.vectorOfNumberToVectorOfPoints(v1), len(v1))
assert len(xAlexa2_IPE_Registration.reg_template) == (4 + len(v1)), " something wrong "
time_end_xAlexa2_IPE_Registration = datetime.datetime.now()
timedelta_xAlexa2_IPE_Registration = \
(time_end_xAlexa2_IPE_Registration - time_start_xAlexa2_IPE_Registration)
#print(timedelta_xAlexa2_IPE_Registration.total_seconds())
#==============================================
time_start_xAlexa2_IPE_Authentication = datetime.datetime.now()
xAlexa2_IPE_Authentication = \
Alexa2_IPE_Authentication(xAlexa2_IPE_Setup.msk, Alexa2_IPE_Setup.vectorOfNumberToVectorOfPoints(v2), len(v2))
assert len(xAlexa2_IPE_Authentication.auth_template) == (4 + len(v2)), " something wrong "
assert len(xAlexa2_IPE_Authentication.auth_template) == len(xAlexa2_IPE_Registration.reg_template), " something wrong "
time_end_xAlexa2_IPE_Authentication = datetime.datetime.now()
timedelta_xAlexa2_IPE_Authentication = \
(time_end_xAlexa2_IPE_Authentication - time_start_xAlexa2_IPE_Authentication)
#print(timedelta_xAlexa2_IPE_Authentication.total_seconds())
#==============================================
time_start_xAlexa2_IPE_Decryption_init = datetime.datetime.now()
xAlexa2_IPE_Decryption = \
Alexa2_IPE_Decryption(optimal_ate, xAlexa2_IPE_Setup.G1, xAlexa2_IPE_Setup.G2, xAlexa2_IPE_Registration.reg_template, xAlexa2_IPE_Authentication.auth_template)
time_end_xAlexa2_IPE_Decryption_init = datetime.datetime.now()
timedelta_xAlexa2_IPE_Decryption_init = \
(time_end_xAlexa2_IPE_Decryption_init - time_start_xAlexa2_IPE_Decryption_init)

time_start_Alexa2_IPE_Decryption_get_pairing_of_templates = datetime.datetime.now()
xAlexa2_IPE_Decryption.get_pairing_of_templates()
time_end_Alexa2_IPE_Decryption_get_pairing_of_templates = datetime.datetime.now()
timedelta_Alexa2_IPE_Decryption_get_pairing_of_templates = \
(time_end_Alexa2_IPE_Decryption_get_pairing_of_templates - time_start_Alexa2_IPE_Decryption_get_pairing_of_templates)

time_start_Alexa2_IPE_Decryption_get_inner_product = datetime.datetime.now()
xAlexa2_IPE_Decryption.get_inner_product()
time_end_Alexa2_IPE_Decryption_get_inner_product = datetime.datetime.now()
timedelta_Alexa2_IPE_Decryption_get_inner_product = \
(time_end_Alexa2_IPE_Decryption_get_inner_product - time_start_Alexa2_IPE_Decryption_get_inner_product)

time_start_Alexa2_IPE_Decryption_get_hamming_dictance = datetime.datetime.now()
xAlexa2_IPE_Decryption.get_hamming_dictance()
time_end_Alexa2_IPE_Decryption_get_hamming_dictance = datetime.datetime.now()
timedelta_Alexa2_IPE_Decryption_get_hamming_dictance = \
(time_end_Alexa2_IPE_Decryption_get_hamming_dictance - time_start_Alexa2_IPE_Decryption_get_hamming_dictance)
timedelta_xAlexa2_IPE_Decryption = \
(timedelta_xAlexa2_IPE_Decryption_init + timedelta_Alexa2_IPE_Decryption_get_pairing_of_templates + \
timedelta_Alexa2_IPE_Decryption_get_inner_product + timedelta_Alexa2_IPE_Decryption_get_hamming_dictance)
#==============================================
time_end2 = datetime.datetime.now()
timedelta2 = (time_end2 - time_start2)

In [20]:
print(" inner product via IPE (improved): ")
print(xAlexa_IPE_Decryption.inner_product)
print(" hamming dictance via IPE (improved): ")
print(xAlexa_IPE_Decryption.hamming_dictance)
print(" inner product via initial vectors : ")
print(Alexa_Hamming.inner_product(v1, v2))
print(" inner product via hamming distance : ")
print(Alexa_Hamming.hamming_distance(v1, v2))

 inner product via IPE (improved): 
-4
 hamming dictance via IPE (improved): 
0.625
 inner product via initial vectors : 
-4
 inner product via hamming distance : 
0.625


In [33]:
print(timedelta2.total_seconds()/60, " (minutes)")
print(timedelta2.total_seconds(), " (seconds)")
print((10**6)*timedelta2.total_seconds(), " (microseconds)")

0.23449675  (minutes)
14.069805  (seconds)
14069805.0  (microseconds)


In [32]:
print("{:<20}".format((timedelta2.total_seconds()/60)), " (minutes)" , "\t vs ", "{:<20}".format((timedelta.total_seconds()/60)), " (minutes)")
print("{:<20}".format(timedelta2.total_seconds()), " (seconds)" , "\t vs ", "{:<20}".format(timedelta.total_seconds()), " (seconds)")
print("{:<20}".format(((10**6)*timedelta2.total_seconds())), " (microseconds)", "\t vs ", "{:<20}".format(((10**6)*timedelta.total_seconds())), " (microseconds)")

0.23449675            (minutes) 	 vs  0.48482773333333334   (minutes)
14.069805             (seconds) 	 vs  29.089664             (seconds)
14069805.0            (microseconds) 	 vs  29089664.0            (microseconds)


In [34]:
print(timedelta_Alexa_IPE_Setup.total_seconds(), " (seconds)")
print(timedelta_Alexa2_IPE_Setup.total_seconds(), " (seconds)")

7.561432  (seconds)
0.053003  (seconds)


In [35]:
print(timedelta_xAlexa_IPE_Registration.total_seconds(), " (seconds)")
print(timedelta_xAlexa2_IPE_Registration.total_seconds(), " (seconds)")

9.168525  (seconds)
3.389194  (seconds)


In [36]:
print(timedelta_xAlexa_IPE_Authentication.total_seconds(), " (seconds)")
print(timedelta_xAlexa2_IPE_Authentication.total_seconds(), " (seconds)")

2.670153  (seconds)
0.941054  (seconds)


In [40]:
print(timedelta_xAlexa_IPE_Decryption.total_seconds(), " (seconds)")
print(timedelta_xAlexa2_IPE_Decryption.total_seconds(), " (seconds)")
# approximate equality of times according to same algorithm

9.688554  (seconds)
9.685554  (seconds)
